<a href="https://colab.research.google.com/github/elahe-ghafari/my-final-project/blob/main/VGG19%2BViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os                       # for working with files

import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import seaborn as sns
import torch                    # Pytorch module
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
import torchvision.transforms as transforms   # for transforming images into tensors
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model
import tensorflow as ts
from  tensorflow import keras
import itertools
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
from zipfile import ZipFile
with ZipFile ('/content/drive/My Drive/melanom.zip','r') as zipObj:
  zipObj.extractall('melanom')

In [ ]:
from google.colab import files
!zip -r /content/melanom.zip /content/melanom
files.download('/content/drive/My Drive/melanom.zip')

In [ ]:
train_dir = 'melanom/melanoma_cancer_dataset/train'
skin = os.listdir(train_dir)
skin

In [ ]:
# Number of images for each diseas
nums_train = {}
nums_val = {}
for s in skin:
    nums_train[s] = len(os.listdir(train_dir + '/' + s))
img_per_class_train = pd.DataFrame(nums_train.values(), index=nums_train.keys(), columns=["no. of images"])
print('Train data distribution :')
img_per_class_train

In [ ]:
plt.figure(figsize=(10,10))
plt.title('data distribution ',fontsize=30)
plt.ylabel('Number of image',fontsize=20)
plt.xlabel('Type of skin cancer',fontsize=20)

keys = list(nums_train.keys())
vals = list(nums_train.values())
sns.barplot(x=keys, y=vals)

In [ ]:
# Function to show image
train = ImageFolder(train_dir, transform=transforms.ToTensor())
def show_image(image, label):
    print("Label :" + train.classes[label] + "(" + str(label) + ")")
    return image.permute(1, 2, 0)

Modeling

In [ ]:
train_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                         rotation_range = 0.30 ,
                                                         horizontal_flip = True ,
                                                         validation_split = 0.2
                                                         )
valid_gen =  keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split = 0.2)
train_data = train_gen.flow_from_directory(train_dir, subset='training', target_size=(224,224), batch_size=64, color_mode='rgb',
                                            class_mode='categorical', shuffle=True)

test_data = valid_gen.flow_from_directory(train_dir, subset='validation', target_size=(224,224), batch_size=64, color_mode='rgb',
                                            class_mode='categorical', shuffle=False)

MODEL vgg19+VIT

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.applications import VGG19
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dense, Dropout

# Define the CNN backbone (you can use pre-trained models such as VGG16)
def cnn_backbone(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = True
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    return Model(base_model.input, x)

# Define the Vision Transformer component
def vision_transformer(input_shape):
    input_layer = Input(shape=input_shape)
    x = Reshape((-1, input_shape[2]))(input_layer)  # Flatten spatial dimensions
    x = LayerNormalization()(x)
    x = MultiHeadAttention(num_heads=8, key_dim=32, dropout=0.1)([x, x])
    x = Dropout(0.1)(x)
    x = LayerNormalization()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.1)(x)
    return Model(input_layer, x)

# Input shape
input_shape = (224, 224, 3)

# Define the CNN backbone
cnn_model = cnn_backbone(input_shape)

# Define the Vision Transformer component
vit_model = vision_transformer(cnn_model.output_shape[1:])

# Combine the CNN backbone and Vision Transformer component
cnn_output = cnn_model.output
vit_output = vit_model(cnn_output)

# Define the final model
final_output = Dense(1, activation="sigmoid")(vit_output) # For binary classification (melanoma or not)
hybrid_model = Model(inputs=cnn_model.input, outputs=final_output)

# Compile the model
opt = Adam(learning_rate=0.0001)
hybrid_model.compile(optimizer=opt, loss=binary_crossentropy, metrics=['accuracy'])

# Summary of the model
hybrid_model.summary()


In [ ]:
import tensorflow as tf
keras.utils.plot_model(
    model_vgg16,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

In [ ]:
import keras
from keras.callbacks import EarlyStopping


# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define other parameters and compile the model
# Assuming you have already defined and compiled your model


# Train the model with early stopping
history = model_vgg16.fit(train_data, epochs=10, validation_data=test_data, callbacks=[early_stopping])

In [ ]:
plt.figure(figsize = (20,5))
plt.subplot(1,2,1)
plt.title("Train and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(history.history['loss'],label="Train Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.xlim(0, 10)
plt.ylim(0.0,1.0)
plt.legend()

plt.subplot(1,2,2)
plt.title("Train and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.plot(history.history['accuracy'], label="Train Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.xlim(0, 9.25)
plt.ylim(0.75,1.0)
plt.legend()
plt.tight_layout()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.metrics import classification_report

Y_pred = model_vgg16.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(test_data.classes, y_pred))

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
# calculating and plotting the confusion matrix
cm1 = confusion_matrix(test_data.classes, y_pred)
plot_confusion_matrix(conf_mat=cm1,show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()